# **Подготовка данных**

Для начала нам необходимо загрузить файлы в виде pandas DataFrame

In [4]:
# Импорт модулей для работы с файлами в Google Colaboratory

from google.colab import files
import io
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [5]:
# Загрузка локальных файлов

uploaded = files.upload()

Saving musae_git_edges.csv to musae_git_edges (6).csv
Saving musae_git_target.csv to musae_git_target (6).csv


In [6]:
# Импорт модуля pandas
import pandas as pd

# Чтение csv-файлов
target = pd.read_csv(io.BytesIO(uploaded['musae_git_target.csv']))
edges = pd.read_csv(io.BytesIO(uploaded['musae_git_edges.csv']))

Создадим функцию для рассчета характеристик параметров датасетов

In [7]:
def scale_values(value):
    uniq_values = sorted(value.unique())
    shape_value = len(uniq_values)
    if value.dtype == object:
        min_value = "-"
        max_value = "-"
    else:
        min_value = pd.Series(uniq_values).min()
        max_value = pd.Series(uniq_values).max()

    return value.dtype, shape_value, min_value, max_value, uniq_values

## **Работа с узлами** *(musae_git_target.csv)*

Посмотрим характеристики набора

In [8]:
target.head()

,id,name,ml_target
0,0,Eiryyy,0
1,1,shawflying,0
2,2,JpMCarrilho,1
3,3,SuhwanCha,0
4,4,sunilangadi2,1


In [9]:
print('Количество записей: {}\nКоличество признаков: {}'.format(target.shape[0], target.shape[1]))

Количество записей: 5000
Количество признаков: 3


In [10]:
scale_values_target = target.apply(scale_values).transpose()
scale_values_target.columns = ["type", "shape", "min", "max", "uniq_values"]
scale_values_target

,type,shape,min,max,uniq_values
id,int64,5000,0,4999,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
name,object,5000,-,-,"[07Gond, 0bman, 0x4f5da2, 0xHJK, 0xchamin, 100..."
ml_target,int64,2,0,1,"[0, 1]"


Для большего разнообразия добавим пользователям дополнительные характеристики: пол и возраст. Согласно данным из открытых источников, женщин и мужчин в IT 34% и 66% соответственно, тогда как возраст разработчиков:
*   18-24 -> 19,8%
*   25-34 -> 63,9%
*   35-44 -> 14,1%
*   45+ -> 2,2%





In [11]:
import random

target["sex"] = random.choices(["0", "1"], weights=[0.66, 0.34], k=target.shape[0]) # Male = 0; Female = 1
target

,id,name,ml_target,sex
0,0,Eiryyy,0,0
1,1,shawflying,0,0
2,2,JpMCarrilho,1,0
3,3,SuhwanCha,0,1
4,4,sunilangadi2,1,1
...,...,...,...,...
4995,4995,nicarq,0,1
4996,4996,Hansiyuan131,1,1
4997,4997,lamabiker,0,1
4998,4998,nikos-glikis,0,0


In [12]:
target["age"] = [random.choices([random.randint(18, 24), random.randint(25, 34), random.randint(35, 44), random.randint(45, 80)], weights=[0.198, 0.639, 0.141, 0.22])[0] for _ in range(target.shape[0])]
target

,id,name,ml_target,sex,age
0,0,Eiryyy,0,0,58
1,1,shawflying,0,0,23
2,2,JpMCarrilho,1,0,31
3,3,SuhwanCha,0,1,26
4,4,sunilangadi2,1,1,26
...,...,...,...,...,...
4995,4995,nicarq,0,1,31
4996,4996,Hansiyuan131,1,1,18
4997,4997,lamabiker,0,1,50
4998,4998,nikos-glikis,0,0,34


Проверяем, что все данные заполнены.

In [13]:
target.isna().sum()

id           0
name         0
ml_target    0
sex          0
age          0
dtype: int64

Сохраняем данные в csv-файл

In [14]:
target.to_csv('/content/drive/My Drive/Colab Notebooks/ml_social_graphs/ml_target.csv')

## **Работа со связями** *(musae_git_edges.csv)*

Посмотрим характеристики набора

In [15]:
edges.head()

,id_1,id_2
0,1,237
1,1,2383
2,3,495
3,3,3358
4,4,2865


In [16]:
print('Количество записей: {}\nКоличество признаков: {}'.format(edges.shape[0], edges.shape[1]))

Количество записей: 10592
Количество признаков: 2


In [17]:
scale_values_edges = edges.apply(scale_values).transpose()
scale_values_edges.columns = ["type", "shape", "min", "max", "uniq_values"]
scale_values_edges

,type,shape,min,max,uniq_values
id_1,int64,3027,1,4998,"[1, 2, 3, 4, 6, 8, 10, 11, 12, 14, 15, 19, 20,..."
id_2,int64,2730,1,4994,"[1, 5, 6, 7, 8, 9, 11, 12, 18, 19, 21, 22, 23,..."


Исходя из описания данных, связи представляют собой подписки пользователей друг на друга, то есть - ребра неориентированного графа. Так как в дальнейшем эти данные будут использоваться для построения ориентированного графа, необходимо добавить несколько строк, обратных имеющимся, чтобы отобразить взаимную подписку пользователей друг на друга. Возьмем 25% строк случайным образом и поменяем названия колонок местами.

In [18]:
reverse_edges = edges.sample(frac=0.25, random_state=1).rename(columns={"id_1": "id_2", "id_2": "id_1"})
reverse_edges.tail()

,id_2,id_1
4968,2273,3139
2615,1144,3712
5065,2281,625
7745,3761,3474
9247,4664,1701


Добавим полученные строки в конец DataFrame

In [19]:
edges = pd.concat([edges, reverse_edges], ignore_index=True)
edges.head()

,id_1,id_2
0,1,237
1,1,2383
2,3,495
3,3,3358
4,4,2865


In [20]:
edges.tail()

,id_1,id_2
13235,3139,2273
13236,3712,1144
13237,625,2281
13238,3474,3761
13239,1701,4664


Начало таблицы осталось прежним, а окончание совпадает с последними строками reverse_edges.

Далее необходимо создать матрицу смежности в виде DataFrame, в котором строки содержат id пользователя-подписчика, id пользователя, на которого подписались, и значение типа bool в зависимости от того, существует ли такая подписка или нет. Для этого добавим в DataFrame существующих связей колонку edge, заполнив ее единицами.

In [21]:
edges["edge"] = 1
edges

,id_1,id_2,edge
0,1,237,1
1,1,2383,1
2,3,495,1
3,3,3358,1
4,4,2865,1
...,...,...,...
13235,3139,2273,1
13236,3712,1144,1
13237,625,2281,1
13238,3474,3761,1


Создадим матрицу смежности в виде DataFrame из всех возможных пар пользователей.

In [22]:
adjacency = pd.DataFrame(target["id"])
adjacency = adjacency.merge(adjacency, how="cross")
adjacency

,id_x,id_y
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
24999995,4999,4995
24999996,4999,4996
24999997,4999,4997
24999998,4999,4998


И объединим новую таблицу с таблицей существующих связей, взяв от результата merge список всех пар id и связи.

In [23]:
adjacency = adjacency.merge(edges, how="left", left_on=["id_x", "id_y"], right_on=["id_1", "id_2"], copy=False)
adjacency = adjacency[['id_x', 'id_y', 'edge']]
adjacency

,id_x,id_y,edge
0,0,0,NaN
1,0,1,NaN
2,0,2,NaN
3,0,3,NaN
4,0,4,NaN
...,...,...,...
25000010,4999,4995,NaN
25000011,4999,4996,NaN
25000012,4999,4997,NaN
25000013,4999,4998,NaN


У пар, данных о которых не было в исходном файле связей, образовалось значение NaN. Его необходимо заменить на 0.

In [24]:
adjacency["edge"] = adjacency["edge"].fillna(0)
adjacency

,id_x,id_y,edge
0,0,0,0.0
1,0,1,0.0
2,0,2,0.0
3,0,3,0.0
4,0,4,0.0
...,...,...,...
25000010,4999,4995,0.0
25000011,4999,4996,0.0
25000012,4999,4997,0.0
25000013,4999,4998,0.0


Проверяем, что количество единиц в полученном DataFrame совпадает с числом существующих связей в edges.

In [25]:
adjacency["edge"].value_counts()[1] == edges.shape[0]

True

Проверяем, что все данные заполнены.

In [26]:
adjacency.isna().sum()

id_x    0
id_y    0
edge    0
dtype: int64

Сохраняем данные в csv-файл

In [27]:
adjacency.to_csv('/content/drive/My Drive/Colab Notebooks/ml_social_graphs/ml_adjacency.csv')

## **Объединение таблиц в один файл**

Объединим данные пользователей с таблицей связей для дальнейшего обучения

In [28]:
df = adjacency.merge(target, how='left', left_on='id_x', right_on='id', copy=False)
df = df.drop('id', axis=1)
df = df.rename(columns={label:label+'_x' for label in target.columns})
df

,id_x,id_y,edge,name_x,ml_target_x,sex_x,age_x
0,0,0,0.0,Eiryyy,0,0,58
1,0,1,0.0,Eiryyy,0,0,58
2,0,2,0.0,Eiryyy,0,0,58
3,0,3,0.0,Eiryyy,0,0,58
4,0,4,0.0,Eiryyy,0,0,58
...,...,...,...,...,...,...,...
25000010,4999,4995,0.0,jeffstokes72,1,0,28
25000011,4999,4996,0.0,jeffstokes72,1,0,28
25000012,4999,4997,0.0,jeffstokes72,1,0,28
25000013,4999,4998,0.0,jeffstokes72,1,0,28


In [29]:
df = df.merge(target, how='left', left_on='id_y', right_on='id', copy=False)
df = df.drop('id', axis=1)
df = df.rename(columns={label:label+'_y' for label in target.columns})
df

,id_x,id_y,edge,name_x,ml_target_x,sex_x,age_x,name_y,ml_target_y,sex_y,age_y
0,0,0,0.0,Eiryyy,0,0,58,Eiryyy,0,0,58
1,0,1,0.0,Eiryyy,0,0,58,shawflying,0,0,23
2,0,2,0.0,Eiryyy,0,0,58,JpMCarrilho,1,0,31
3,0,3,0.0,Eiryyy,0,0,58,SuhwanCha,0,1,26
4,0,4,0.0,Eiryyy,0,0,58,sunilangadi2,1,1,26
...,...,...,...,...,...,...,...,...,...,...,...
25000010,4999,4995,0.0,jeffstokes72,1,0,28,nicarq,0,1,31
25000011,4999,4996,0.0,jeffstokes72,1,0,28,Hansiyuan131,1,1,18
25000012,4999,4997,0.0,jeffstokes72,1,0,28,lamabiker,0,1,50
25000013,4999,4998,0.0,jeffstokes72,1,0,28,nikos-glikis,0,0,34


Имя пользователя не влияет на подписки, для вывода результатов же мы можем получить его по id, поэтому удаляем name_x и name_y

In [30]:
df = df.drop(['name_x', 'name_y'], axis=1)
df.head()

,id_x,id_y,edge,ml_target_x,sex_x,age_x,ml_target_y,sex_y,age_y
0,0,0,0.0,0,0,58,0,0,58
1,0,1,0.0,0,0,58,0,0,23
2,0,2,0.0,0,0,58,1,0,31
3,0,3,0.0,0,0,58,0,1,26
4,0,4,0.0,0,0,58,1,1,26


Проверяем, что все данные заполнены.

In [31]:
df.isna().sum()

id_x           0
id_y           0
edge           0
ml_target_x    0
sex_x          0
age_x          0
ml_target_y    0
sex_y          0
age_y          0
dtype: int64

Посмотрим характеристики набора

In [32]:
print('Количество записей: {}\nКоличество признаков: {}'.format(df.shape[0], df.shape[1]))

Количество записей: 25000015
Количество признаков: 9


In [33]:
scale_values_df = df.apply(scale_values).transpose()
scale_values_df.columns = ["type", "shape", "min", "max", "uniq_values"]
scale_values_df

,type,shape,min,max,uniq_values
id_x,int64,5000,0,4999,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
id_y,int64,5000,0,4999,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
edge,float64,2,0.0,1.0,"[0.0, 1.0]"
ml_target_x,int64,2,0,1,"[0, 1]"
sex_x,object,2,-,-,"[0, 1]"
age_x,int64,63,18,80,"[18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 2..."
ml_target_y,int64,2,0,1,"[0, 1]"
sex_y,object,2,-,-,"[0, 1]"
age_y,int64,63,18,80,"[18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 2..."


Сохраняем данные в csv-файл

In [34]:
df.to_csv('/content/drive/My Drive/Colab Notebooks/ml_social_graphs/dataframe.csv')